In [1]:
import pandas as pd
import tkinter
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow import keras

In [4]:
import random
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from numpy import asarray
from sklearn.preprocessing import PolynomialFeatures
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import math

## def Subir Data

In [5]:
def subirData():
    global data, variables
    direccion=input("coloque direccion de la data")+"\\"+input("cual es el nombre del archivo?")
    print("")
    if "csv" in direccion:
        data=pd.read_csv(direccion)
    elif "xlsx"in direccion:
        data=pd.read_excel(direccion)
    print("la base de datos tiene", data.shape[1], "columnas y", data.shape[0], "filas")
    data.dropna(inplace= True)
    data=data.reset_index(drop=True)
    print("cantidad de filas completas es", data.shape[0])
    print("")
    variables=[]
    for i in data:
        variables.append(i)

In [6]:
def matarOutlayers():
    global data
    filasCompletas=data.shape[0]
    for i in variables:
        if data[i].dtypes != ("object"):
            Q1=data[i].quantile(0.25)
            Q3=data[i].quantile(0.75)
            IQR=Q3-Q1
            Lower_Whisker = Q1 - 1.5 * IQR
            Upper_Whisker = Q3 + 1.5 * IQR
            data=data[data[i] < Upper_Whisker]
            data=data[data[i] > Lower_Whisker]
    print("hecho, outlayers removidos")
    print("hay un total de", filasCompletas-data.shape[0],"outliers removidos")
    print("las filas útiles son",data.shape[0])
    print("")
    

In [7]:
def limpiarData():
    global data
    if input("Quitar Outlayers? ") == "si":
        matarOutlayers()
    else:
        print("okey")
        print("")
    display(data.sample(7))

## def Variables

In [8]:
def elegirVariables():
    global ind, dep, data, variables, j, outputData, inputData
    print("puede definir variables independientes y dependientes entre las siguientes:")
    print(variables)
    print("")
    frame = tkinter.Frame()
    j=0
    ind=[]
    dep=[]
    def derecha():
        global j
        j=j+1
        v.set(variables[j])
    def izquierda():
        global j
        j=j-1
        v.set(variables[j])
    def independiente():
        global ind
        ind.append(variables[j])
        print(variables[j], "es una variable independiente")
    def dependiente():
        global dep
        dep.append(variables[j])
        print(variables[j], "es una variable dependiente")
    print("")
    botonizquierda=tkinter.Button(frame, text="<<<", command=izquierda)
    botonizquierda.pack(side=tkinter.LEFT)
    v = tkinter.StringVar()
    v.set(variables[j])
    text = tkinter.Entry(frame, textvariable=v )
    text.pack(side=tkinter.LEFT);
    botonderecha=tkinter.Button(frame, text=">>>", command=derecha)
    botonderecha.pack(side=tkinter.LEFT)
    botonindependiente=tkinter.Button(frame, text="Independiente", command=independiente)
    botonindependiente.pack(side=tkinter.LEFT)
    botondependiente=tkinter.Button(frame, text="Dependiente", command=dependiente)
    botondependiente.pack(side=tkinter.LEFT)
    botoncerrar=tkinter.Button(frame, text="Cerrar", command=frame.destroy).pack()
    frame.pack()
    frame.mainloop()
    finalVariables=ind+dep
    data=data[finalVariables]
    display(data.sample(7))
    print("")
    outputData=data[dep]
    inputData=data[ind]

## def Preprocessing

In [9]:
def dummies():
    global inputData, outputData, ind, dep
    for i in inputData:
        if inputData[i].dtype == "object":
            print("creamos dummy variables para la variable", i)
            add=pd.get_dummies(inputData[i])
            for j in add:
                inputData[str(str(i)+" - "+str(j))]=add[j]
                ind.append(str(str(i)+" - "+str(j)))
            ind.remove(i)
            inputData=inputData.drop(columns=i)
    for i in outputData:
        if outputData[i].dtype == "object":
            print("creamos dummy variables para la variable", i)
            add=pd.get_dummies(outputData[i])
            for j in add:
                outputData[str(str(i)+" - "+str(j))]=add[j]
                dep.append(str(str(i)+" - "+str(j)))
            dep.remove(i)
            outputData=outputData.drop(columns=i)
    print("Input Data")
    display(inputData.sample(7))
    print("")
    print("Output Data")
    display(outputData.sample(7))
    print("")

In [10]:
def razaModelo():
    global outputData
    unOutcome=False
    menosQueCero=False
    masQueUno=False
    intermedio=False
    algunoMaroyAUno=False
    masDeUnLabel=False
    if outputData.shape[1] == 1:
        unOutcome=True
        for i in outputData[dep[0]]:
            if i < 0:
                menosQueCero=True
            elif i > 1:
                masQueUno=True
            elif not(i==1 or i==0):
                intermedio=True
    else:
        for i in dep:
            if sum(outputData[dep[0]]>1*1) > 0:
                algunoMaroyAUno=True
            else:
                for i in range(outputData.shape[0]):
                    if sum(np.array(outputData.loc[[i]])[0]) > 1:
                        masDeUnLabel=True
    if masDeUnLabel:
        return "multiple label"
    elif algunoMaroyAUno:
        return "multiple regression"
    elif menosQueCero:
        return "regresion arbitraria"
    elif masQueUno:
        return "regresion positiva"
    elif intermedio:
        return "regresion porcentual"
    elif unOutcome:
        return "clasificacion binaria"
    else:
        return "simple label"

In [34]:
def standarizarData():
    global inputData, ind, inputTrain, inputTest, outputTrain, outputTest, outputData, clasReg, subZero, minimo, data, scaler
    if input("Standarizar data? ") == "si":    
        scaler = preprocessing.StandardScaler().fit(inputData)
        inputData = scaler.transform(inputData)
        inputData = pd.DataFrame(inputData, columns = ind)
        print("hecho, data standarizada")
        print("")
    else:
        print("okey")
        print("")
    clasReg = False
    subZero = False
    for i in outputData[dep[0]]:
        if i != True:
            if i != False:
                clasReg=True
        if i < 0:
            subZero = True
    if raza == "clasificacion binaria":
        print("casos totales", data.shape[0])
        print("casos positivos",sum(data[dep[0]]))
        print("casos negativos",data.shape[0]-sum(data[dep[0]]))
        minimo=min((data.shape[0]-sum(data[dep[0]])),(sum(data[dep[0]])))
        print("proporcion de negativos sobre positivos",sum(data[dep[0]])/data.shape[0])
        print("")
        if input("Equilibrar la cantidad de resultados positivos y negativos? ") == "si":
            ceros=0
            unos=0
            for i in range(data.shape[0]):
                if data[dep[0]][i]==0:
                    ceros+=1
                    if ceros > minimo:
                        data=data.drop(i, axis=0)
                        inputData=inputData.drop(i, axis=0)
                        outputData=outputData.drop(i, axis=0)
                elif data[dep[0]][i]==1:
                    unos+=1
                    if unos > minimo:
                        data=data.drop(i, axis=0)
                        inputData=inputData.drop(i, axis=0)
                        outputData=outputData.drop(i, axis=0)
            print("casos totales", data.shape[0])
            print("casos positivos",sum(data[dep[0]]))
            print("casos negativos",data.shape[0]-sum(data[dep[0]]))
            minimo=min((data.shape[0]-sum(data[dep[0]])),(sum(data[dep[0]])))
            print("nueva proporcion de negativos sobre positivos",sum(data[dep[0]])/data.shape[0])
            print("")
    inputTrain, inputTest, outputTrain, outputTest = train_test_split(inputData, outputData, test_size=0.33, random_state=14)
    print("Input Data")
    display(inputData.sample(7))
    print("")

## def Modelos

In [12]:
def runCTree(j):
    if (raza == "clasificacion binaria") or (raza == "multiple label") or (raza == "simple label"):
        desicion_tree = tree.DecisionTreeClassifier(max_depth=j).fit(inputTrain, outputTrain)
        print("accuracy score:", round(accuracy_score(outputTest, desicion_tree.predict(inputTest)),4))
        shw(desicion_tree,0)
        return [desicion_tree, accu(desicion_tree,0), "tree", 0]
    elif (raza == "regresion arbitraria") or (raza == "regresion porcentual") or (raza == "regresion positiva") or (raza == "multiple regression"):
        desicion_tree = tree.DecisionTreeRegressor(max_depth=j).fit(inputTrain, outputTrain)
        print("mean squared error:", round(MeanSquaredError()(outputTest,desicion_tree.predict(inputTest)).numpy(),4))
        shw(desicion_tree,0)
        return [desicion_tree, accu(desicion_tree,0), "tree", 0]

In [13]:
def runRForest(j):
    if (raza == "clasificacion binaria") or (raza == "multiple label") or (raza == "simple label"):
        random_forest = RandomForestClassifier(max_depth=j).fit(inputTrain, outputTrain)
        print("accuracy score:", round(accuracy_score(outputTest, random_forest.predict(inputTest)),4))
        shw(random_forest,0)
        return [random_forest, accu(random_forest,0), "forest", 0]
    elif (raza == "regresion arbitraria") or (raza == "regresion porcentual") or (raza == "regresion positiva") or (raza == "multiple regression"):
        random_forest = tree.DecisionTreeRegressor(max_depth=j).fit(inputTrain, outputTrain)
        print("mean squared error:", round(MeanSquaredError()(outputTest,random_forest.predict(inputTest)).numpy(),4))
        shw(random_forest,0)
        return [random_forest, accu(random_forest,0), "forest", 0]

In [14]:
def runPoly(ii):
    global inputTrain, inputTest, outputTrain, outputTest, clasReg, modelosTodosPoly
    inputTrainPoly=PolynomialFeatures(degree=ii).fit_transform(inputTrain)
    inputTestPoly=PolynomialFeatures(degree=ii).fit_transform(inputTest)
    lineaRegPoly=LinearRegression().fit(inputTrainPoly, outputTrain)
    print("coefficient of determination:", round(lineaRegPoly.score(inputTrainPoly, outputTrain),4)*100, "%")
    shw(lineaRegPoly,ii)
    accur=accu(lineaRegPoly,ii)
    print("mse or accuracy:", round(accur,4))
    return [lineaRegPoly, accur, "poly", ii]

In [15]:
def disenarModelos():
    global layers, hidden1, activation1, hidden2, activation2, hidden3, activation3, optimizer, learningRate
    layers, hidden1, activation1, hidden2, activation2, hidden3, activation3, optimizer, learningRate=0,0,0,0,0,0,0,0,0
    layers=(input("cuantas capas tendrá el modelo? (1, 2 o 3) ")[::].split(",")) #1,2,3
    for i in range(len(layers)):
        layers[i]=int(layers[i])
    hidden1=(input("cuantas celulas habrá en la 1er capa? ")[::].split(","))
    for i in range(len(hidden1)):
        hidden1[i]=int(hidden1[i])
    activation1=(input("activacion de la capa 1? (sigmoid,softmax,tanh,relu,softplus,softsign,selu,elu,exponential) ")[::].split(",")) #sigmoid,softmax,tanh,relu,softplus,softsign,selu,elu,exponential
    if 2 in layers or 3 in layers:
        hidden2=(input("cuantas celulas habrá en la 2da capa? ")[::].split(","))
        for i in range(len(hidden2)):
            hidden2[i]=int(hidden2[i])
        activation2=(input("activacion de la capa 2? (sigmoid,softmax,tanh,relu,softplus,softsign,selu,elu,exponential) ")[::].split(",")) #sigmoid,softmax,tanh,relu,softplus,softsign,selu,elu,exponential
    if 3 in layers:
        hidden3=(input("cuantas celulas habrá en la 3er capa? ")[::].split(","))
        for i in range(len(hidden3)):
            hidden3[i]=int(hidden3[i])
        activation3=(input("activacion de la capa 3? (sigmoid,softmax,tanh,relu,softplus,softsign,selu,elu,exponential) ")[::].split(",")) #sigmoid,softmax,tanh,relu,softplus,softsign,selu,elu,exponential
    optimizer=[Adam] #Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
    learningRate=(input("learning rate? (0.1, 0.01, 0.001, etc) ")[::].split(",")) #0.1, 0.01, 0.001
    for i in range(len(learningRate)):
        learningRate[i]=float(learningRate[i])
    print("")

In [16]:
def crearModelos():
    global models, optimizer, learningRate, layers, hidden1, activation1, hidden2, activation2, hidden3, activation3
    models=[]
    for i in optimizer:
        for j in learningRate:
            for k in layers:
                if k==1:
                    for m in hidden1:
                        for n in activation1:
                            models.append((m,n,0,0,0,0,i,j))
                elif k==2:
                    for m in hidden1:
                        for n in activation1:
                            for o in hidden2:
                                for p in activation2:
                                    models.append((m,n,o,p,0,0,i,j))
                elif k==3:
                    for m in hidden1:
                        for n in activation1:
                            for o in hidden2:
                                for p in activation2:
                                    for q in hidden3:
                                        for r in activation3:
                                            models.append((m,n,o,p,q,r,i,j))

In [17]:
def run(h1,a1,h2,a2,h3,a3,optimizer,lr):
    global clasReg, subZero
    print("")
    print(h1,a1,h2,a2,h3,a3,optimizer,lr)
    print("")
    model=Sequential()
    model.add(Dense(h1, input_shape=(inputData.shape[1],), activation=a1))
    if h2 > 0:
        model.add(Dense(h2, activation=a2))
    if h3 > 0:
        model.add(Dense(h3, activation=a3))
    early=EarlyStopping(monitor='val_loss', patience=5)
    if raza == "regresion arbitraria":
        model.add(Dense(outputData.shape[1]))
        model.compile(optimizer (learning_rate = lr), loss="mean_squared_error", metrics=['mae', 'mse'])
        print("regresion arbitraria - activación linear en la última capa")
        print("")
        model.fit(inputTrain, outputTrain, validation_data=(inputTest, outputTest), epochs=1000, callbacks=[early], verbose=0)
        shw(model,0)
        print("mean squared error:", round(MeanSquaredError()(outputTest,model.predict(inputTest)).numpy(),4))
        return [model, accu(model,0), "neural", 0]
    elif raza == "regresion porcentual":
        model.add(Dense(outputData.shape[1], activation="sigmoid"))
        model.compile(optimizer (learning_rate = lr), loss="mean_squared_error", metrics=['mae', 'mse'])
        print("regresion porcentual - activación sigmoid en la última capa")
        print("")
        model.fit(inputTrain, outputTrain, validation_data=(inputTest, outputTest), epochs=1000, callbacks=[early], verbose=0)
        shw(model,0)
        print("mean squared error:", round(MeanSquaredError()(outputTest,model.predict(inputTest)).numpy(),4))
        return [model, accu(model,0), "neural", 0]
    elif raza == "regresion positiva":
        model.add(Dense(outputData.shape[1], activation="relu"))
        model.compile(optimizer (learning_rate = lr), loss="mean_squared_error", metrics=['mae', 'mse'])
        print("regresion positiva - activación relu en la última capa")
        print("")
        model.fit(inputTrain, outputTrain, validation_data=(inputTest, outputTest), epochs=1000, callbacks=[early], verbose=0)
        shw(model,0)
        print("mean squared error:", round(MeanSquaredError()(outputTest,model.predict(inputTest)).numpy(),4))
        return [model, accu(model,0), "neural", 0]
    elif raza == "multiple regression":
        model.add(Dense(outputData.shape[1]))
        model.compile(optimizer (learning_rate = lr), loss="mean_squared_error", metrics=['mae', 'mse'])
        print("multiple regression - activación linear en la última capa")
        print("")
        model.fit(inputTrain, outputTrain, validation_data=(inputTest, outputTest), epochs=1000, callbacks=[early], verbose=0)
        shw(model,0)
        print("mean squared error:", round(MeanSquaredError()(outputTest,model.predict(inputTest)).numpy(),4))
        return [model, accu(model,0), "neural", 0]
    elif raza == "clasificacion binaria":
        model.add(Dense(outputData.shape[1], activation="sigmoid"))
        model.compile(optimizer (learning_rate = lr), loss="binary_crossentropy", metrics=['accuracy'])
        print("clasificacion binaria - activación sigmoid en la última capa")
        print("")
        model.fit(inputTrain, outputTrain, validation_data=(inputTest, outputTest), epochs=1000, callbacks=[early], verbose=0)
        shw(model,0)
        print("accuracy score:", round(accuracy_score(outputTest, (model.predict(inputTest)>0.5)*1),4))
        return [model, accu(model,0), "neural", 0]
    elif raza == "multiple label":
        model.add(Dense(outputData.shape[1], activation="sigmoid"))
        model.compile(optimizer (learning_rate = lr), loss="binary_crossentropy", metrics=['accuracy'])
        print("multiple label - activación sigmoid en la última capa")
        print("")
        model.fit(inputTrain, outputTrain, validation_data=(inputTest, outputTest), epochs=1000, callbacks=[early], verbose=0)
        shw(model,0)
        print("accuracy score:", round(accuracy_score(outputTest, (model.predict(inputTest)>0.5)*1),4))
        return [model, accu(model,0), "neural", 0]
    elif raza == "simple label":
        model.add(Dense(outputData.shape[1], activation="softmax"))
        model.compile(optimizer (learning_rate = lr), loss="categorical_crossentropy", metrics=['accuracy'])
        print("multiple label - activación softmax en la última capa")
        print("")
        model.fit(inputTrain, outputTrain, validation_data=(inputTest, outputTest), epochs=1000, callbacks=[early], verbose=0)
        shw(model,0)
        print("accuracy score:", round(accuracy_score(outputTest, (model.predict(inputTest)>0.5)*1),4))
        return [model, accu(model,0), "neural", 0]
    print("")

## def Evaluar Modelos

In [18]:
def accu(model,g):
    global outputTest, inputTest
    if g==0:
        if (raza == "regresion arbitraria") or (raza == "regresion porcentual") or (raza == "regresion positiva") or (raza == "multiple regression"):
            return MeanSquaredError()(outputTest,model.predict(inputTest)).numpy()
        elif (raza == "clasificacion binaria") or (raza == "simple label") or (raza == "multiple label"):
            return accuracy_score(outputTest, (model.predict(inputTest)>0.5)*1)
        else:
            print("raza =", raza)
    else:
        if (raza == "regresion arbitraria") or (raza == "regresion porcentual") or (raza == "regresion positiva") or (raza == "multiple regression"):
            return MeanSquaredError()(outputTest,model.predict(PolynomialFeatures(degree=g).fit_transform(inputTest))).numpy()
        elif (raza == "clasificacion binaria") or (raza == "simple label") or (raza == "multiple label"):
            return accuracy_score(outputTest, (model.predict(PolynomialFeatures(degree=g).fit_transform(inputTest))>0.5)*1)
        else:
            print("raza =", raza)

In [19]:
def shw(model,g):
    global inputTest, dep, outputTest
    if g == 0:
        if (raza == "regresion arbitraria") or (raza == "regresion positiva") or (raza == "regresion porcentual"):
            show=pd.DataFrame(model.predict(inputTest), columns = ["predicted"])
            show["actual"]=outputTest.reset_index(drop=True, inplace=False)
            show["actual"]=show["actual"].fillna(0)
            display(show.sample(7))
        elif raza == "multiple regression":
            predDep=[]
            actDep=[]
            for i in dep:
                predDep.append("pred-"+i)
                actDep.append("act-"+i)
            show=pd.DataFrame(model.predict(inputTest), columns = predDep)
            show[actDep]=outputTest.reset_index(drop=True, inplace=False)
            show[actDep]=show[actDep].fillna(0)
            display(show.sample(7))
        elif raza == "clasificacion binaria":
            show=pd.DataFrame(model.predict(inputTest), columns = ["predicted"])
            show["result"]=(model.predict(inputTest)>0.5)*1
            show["actual"]=outputTest.reset_index(drop=True, inplace=False)
            show["actual"]=show["actual"].fillna(0)
            show["actual"]=show["actual"].astype(int)
            display(show.sample(7))
        elif (raza == "multiple label") or (raza == "simple label"):
            predDep=[]
            actDep=[]
            resDep=[]
            for i in dep:
                predDep.append("pred-"+i)
                actDep.append("act-"+i)
                resDep.append("res-"+i)
            show=pd.DataFrame(model.predict(inputTest), columns = predDep)
            show[resDep]=(model.predict(inputTest)>0.5)*1
            show[actDep]=outputTest.reset_index(drop=True, inplace=False)
            show[actDep]=show[actDep].fillna(0)
            display(show.sample(7))
    else:
        if (raza == "regresion arbitraria") or (raza == "regresion positiva") or (raza == "regresion porcentual"):
            show=pd.DataFrame(model.predict(PolynomialFeatures(degree=g).fit_transform(inputTest)), columns = ["predicted"])
            show["actual"]=outputTest.reset_index(drop=True, inplace=False)
            show["actual"]=show["actual"].fillna(0)
            display(show.sample(7))
        elif raza == "clasificacion binaria":
            show=pd.DataFrame(model.predict(PolynomialFeatures(degree=g).fit_transform(inputTest)), columns = ["predicted"])
            show["result"]=(model.predict(PolynomialFeatures(degree=g).fit_transform(inputTest))>0.5)*1
            show["actual"]=outputTest.reset_index(drop=True, inplace=False)
            show["actual"]=show["actual"].fillna(0)
            show["actual"]=show["actual"].astype(int)
            display(show.sample(7))

In [20]:
def pando(model,g,inp):
    if g==0:
        return model.predict(inp)
    else:
        return model.predict(PolynomialFeatures(degree=g).fit_transform(inp))

## def Estrategia Automatica

In [21]:
def estAut():
    global modelosTodos, ind, layers, hidden1, activation1, hidden2, activation2, hidden3, activation3, optimizer, learningRate, models, raza
    modelosTodos=[]
    for i in range(1,min(len(ind),15)+1):
        print("")
        print("###################################################################")
        print("---CUSTOM-TREE--------------Model n°", (len(modelosTodos)), "----------------------------")
        print("###################################################################")
        modelosTodos.append(runCTree(i))
        print("")
    for i in range(1,min(len(ind),15)+1):
        print("")
        print("###################################################################")
        print("---RANDOM-FOREST------------Model n°", (len(modelosTodos)), "----------------------------")
        print("###################################################################")
        modelosTodos.append(runRForest(i))
        print("")
    if not((raza=="multiple label") or (raza=="multiple regression") or (raza=="simple label")):
        for i in range(1,min(len(ind),5)+1):
            print("")
            print("###################################################################")
            print("---POLYNOMIAL---------------Model n°", (len(modelosTodos)), "----------------------------")
            print("###################################################################")
            modelosTodos.append(runPoly(i))
            print("")
    layers=[1, 2, 3]
    hidden1=[2, 4, 8, 16, 32, 64]
    activation1=['sigmoid', 'softmax', 'tanh', 'relu', 'softplus', 'softsign', 'selu', 'elu', 'exponential']
    hidden2=[2, 4, 8, 16, 32, 64]
    activation2=['sigmoid', 'softmax', 'tanh', 'relu', 'softplus', 'softsign', 'selu', 'elu', 'exponential']
    hidden3=[2, 4, 8, 16, 32, 64]
    activation3=['sigmoid', 'softmax', 'tanh', 'relu', 'softplus', 'softsign', 'selu', 'elu', 'exponential']
    optimizer=[Adam]
    learningRate=[0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
    crearModelos()
    print("hay un total de",len(models), "modelos")
    print("")
    random.shuffle(models)
    print("-------------------------Modelos-------------------------")
    for i in range(15):
        print(models[i])
    print("")
    for i in range(25):
        print("")
        print("###################################################################")
        print("---DEEP-NEURAL--------------Model n°", (len(modelosTodos)), "----------------------------")
        print("###################################################################")
        modelosTodos.append(run(models[i][0],models[i][1],models[i][2],models[i][3],models[i][4],models[i][5],models[i][6],models[i][7]))
        print("")
    pd.set_option('display.max_rows', pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"]).shape[0]+1)
    display(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"]))
    if (raza == "regresion arbitraria") or (raza == "regresion porcentual") or (raza == "regresion positiva") or (raza == "multiple regression"):
        print(modelosTodos[list(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"]).index(min(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"]))])
        print("modelo n°", list(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"]).index(min(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"])))
    elif (raza == "clasificacion binaria") or (raza == "simple label") or (raza == "multiple label"):
        print(modelosTodos[list(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"]).index(max(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"]))])
        print("modelo n°", list(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"]).index(max(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"])))
    elegirMejorModelo()
    if input("guardar el mejor modelo para usarlo en el futuro? ") == "si":
        guardarMejorModelo()

## Mejor Modelo y Resultados

In [22]:
def elegirMejorModelo():
    global modelosTodos, raza, dep, inputTest, outputTest, mejorModelo
    if (raza == "clasificacion binaria") or (raza == "multiple label") or (raza == "simple label"):
        mejorModelo={"modelo":0, "puntaje":0, "tipo":0, "grado":0}
        print("el mejor modelo es el que aumenta la precisión")
        print("")
        for i, j, k, l in modelosTodos:
            if j > mejorModelo["puntaje"]:
                mejorModelo["modelo"]=i
                mejorModelo["puntaje"]=j
                mejorModelo["tipo"]=k
                mejorModelo["grado"]=l
    else:
        mejorModelo={"modelo":999999999999999999, "puntaje":999999999999999999, "tipo":999999999999999999, "grado":999999999999999999}
        print("el mejor modelo es el que minimiza el Mean Squared Error")
        print("")
        for i, j, k, l in modelosTodos:
            if j < mejorModelo["puntaje"]:
                mejorModelo["modelo"]=i
                mejorModelo["puntaje"]=j
                mejorModelo["tipo"]=k
                mejorModelo["grado"]=l
    print(mejorModelo)
    if (raza == "clasificacion binaria") or (raza == "multiple label") or (raza == "simple label"):
        shw(mejorModelo["modelo"],mejorModelo["grado"])
        print("accuracy:", accu(mejorModelo["modelo"],mejorModelo["grado"]))
        if raza == "clasificacion binaria":
            confusion(mejorModelo["modelo"],mejorModelo["grado"])
    else:
        shw(mejorModelo["modelo"],mejorModelo["grado"])
        print("MSE:", accu(mejorModelo["modelo"],mejorModelo["grado"]))
        print("")

In [124]:
def confusion(model,g):
    inputTest, outputTest
    if g==0:
        confu = confusion_matrix(outputTest,model.predict(inputTest))
    else:
        confu = confusion_matrix(outputTest,(model.predict(PolynomialFeatures(degree=g).fit_transform(inputTest))).astype(int))
    print(confu)
    print("acierta", round(confu[0][0]/sum(confu[0]),2),"de los negativos")
    print("acierta", round(confu[1][1]/sum(confu[1]),2),"de los positivos")
    print(round(confu[0][0]/(confu[0][0]+confu[1][0]),2),"de los predichos como negativos estan okey")
    print(round(confu[1][1]/(confu[1][1]+confu[0][1]),2),"de los predichos como positivos estan okey")
    print("")

In [125]:
def confusion2(model,g):
    inputTest, outputTest
    if g==0:
        confu = confusion_matrix(outputTest,(model.predict(inputTest)>0)*1)
    else:
        confu = confusion_matrix(outputTest,(model.predict(PolynomialFeatures(degree=g).fit_transform(inputTest))).astype(int))
    print(confu)
    print("acierta", round(confu[0][0]/sum(confu[0]),2),"de los negativos")
    print("acierta", round(confu[1][1]/sum(confu[1]),2),"de los positivos")
    print(round(confu[0][0]/(confu[0][0]+confu[1][0]),2),"de los predichos como negativos estan okey")
    print(round(confu[1][1]/(confu[1][1]+confu[0][1]),2),"de los predichos como positivos estan okey")
    print("")

In [24]:
def abrirModelo():
    global mejorModelo
    mejorModelo={}
    mejorModelo["puntaje"] = pd.read_csv("g_puntajetipogradoMejorModelo.csv")["Puntaje"][0]
    mejorModelo["tipo"] = pd.read_csv("g_puntajetipogradoMejorModelo.csv")["Puntaje"][1]
    mejorModelo["grado"] = pd.read_csv("g_puntajetipogradoMejorModelo.csv")["Puntaje"][2]
    mejorModelo["modelo"] = keras.models.load_model('g_mejorModelo.h5')

In [25]:
def guardarMejorModelo():
    global mejorModelo
    mejorModelo["modelo"].save('g_mejorModelo.h5')
    pandas_puntajeTipoGrado=pd.DataFrame([mejorModelo["puntaje"], mejorModelo["tipo"], mejorModelo["grado"]], columns=["Puntaje"])
    pandas_puntajeTipoGrado.to_csv("g_puntajetipogradoMejorModelo.csv")

## Subir Data y elegir Input y Output

In [159]:
if input('desea crear un nuevo modelo?') == "si":
    subirData()
    limpiarData()
    elegirVariables()
    dummies()
    raza = razaModelo()
    standarizarData()
    if input("correr estrategia automática? ") == "si":
        estAut()
else:
    abrirModelo()

desea crear un nuevo modelo?si
coloque direccion de la dataC:\Users\joseg\Downloads
cual es el nombre del archivo?data_catriel (5).csv

la base de datos tiene 34 columnas y 817 filas
cantidad de filas completas es 817

Quitar Outlayers? no
okey



,Unnamed: 0,url,comentarios,comentarios_scaled_ok,comentarios_btm,likes,likes_scaled_ok,likes_btm,up_date,agno,...,Repost,mountainbike,mondrakerbikes,podiumcarbon,enjoy,smile,mmrfactoryracingteam,btt,rotor,pasion
810,810,https://www.instagram.com/p/rA3zU5EWCc/,9,-0.166707,1,124,-1.220255,1,2014-07-28 20:44:34,2014,...,0,0,0,0,0,0,0,0,0,0
565,565,https://www.instagram.com/p/BOiMtuygXnN/,17,-0.076317,1,871,-0.477489,1,2016-12-27 17:41:03,2016,...,0,0,0,0,0,0,0,0,0,0
192,192,https://www.instagram.com/p/BmDuXcbBFhq/,12,-0.132811,0,824,-0.524222,0,2018-08-04 10:12:06,2018,...,1,0,0,0,0,0,0,0,0,0
153,153,https://www.instagram.com/p/BtB7lTrHlXX/,9,-0.166707,0,1618,0.265277,1,2019-01-24 17:08:52,2019,...,0,0,0,0,0,0,0,0,0,0
216,216,https://www.instagram.com/p/BjnCOwfg33j/,80,0.635500,1,4075,2.708351,1,2018-06-04 13:44:55,2018,...,0,0,0,0,0,0,0,0,0,0
637,637,https://www.instagram.com/p/BGsCzCIkWDn/,5,-0.211902,0,406,-0.939853,0,2016-06-15 17:15:38,2016,...,0,0,0,0,0,0,0,0,0,0
472,472,https://www.instagram.com/p/BSHMMMrAZTU/,11,-0.144109,1,762,-0.585871,0,2017-03-26 16:02:33,2017,...,0,0,1,0,0,0,0,0,0,0


puede definir variables independientes y dependientes entre las siguientes:
['Unnamed: 0', 'url', 'comentarios', 'comentarios_scaled_ok', 'comentarios_btm', 'likes', 'likes_scaled_ok', 'likes_btm', 'up_date', 'agno', 'mes', 'dia', 'hora', 'minuto', 'dia_semana', 'len_caption', 'es_video', 'tiene_audio', 'height', 'width', 'tag_users', 'cantidad_hash', 'ubicacion', 'ciclismo', 'Repost', 'mountainbike', 'mondrakerbikes', 'podiumcarbon', 'enjoy', 'smile', 'mmrfactoryracingteam', 'btt', 'rotor', 'pasion']


likes es una variable dependiente
agno es una variable independiente
mes es una variable independiente
dia es una variable independiente
hora es una variable independiente
dia_semana es una variable independiente
len_caption es una variable independiente
es_video es una variable independiente
tiene_audio es una variable independiente
height es una variable independiente
width es una variable independiente
tag_users es una variable independiente
cantidad_hash es una variable independient

,agno,mes,dia,hora,dia_semana,len_caption,es_video,tiene_audio,height,width,tag_users,cantidad_hash,ubicacion,likes
428,2017,5,6,9,sabado,565,0,0,1080,1080,3,2,0,1145
460,2017,4,4,13,martes,336,0,0,1350,1080,1,0,1,1589
452,2017,4,7,10,viernes,187,0,0,810,1080,1,0,1,787
331,2017,8,21,10,lunes,179,0,0,810,1080,5,0,1,967
611,2016,8,16,9,martes,111,0,0,810,1080,0,0,0,589
310,2017,9,10,3,domingo,127,0,0,718,1080,4,0,1,2347
181,2018,9,2,8,domingo,497,0,0,1080,1080,0,1,1,1597



creamos dummy variables para la variable dia_semana
Input Data


...

mean squared error: 498070.6



,model,mse/accuracy,type,grade
0,DecisionTreeRegressor(max_depth=1),1.457129e+06,tree,0
1,DecisionTreeRegressor(max_depth=2),1.631628e+06,tree,0
2,DecisionTreeRegressor(max_depth=3),1.645930e+06,tree,0
3,DecisionTreeRegressor(max_depth=4),1.686347e+06,tree,0
4,DecisionTreeRegressor(max_depth=5),1.685993e+06,tree,0
5,DecisionTreeRegressor(max_depth=6),1.743059e+06,tree,0
6,DecisionTreeRegressor(max_depth=7),1.767084e+06,tree,0
7,DecisionTreeRegressor(max_depth=8),1.789308e+06,tree,0
8,DecisionTreeRegressor(max_depth=9),1.807872e+06,tree,0
9,DecisionTreeRegressor(max_depth=10),1.804136e+06,tree,0


[LinearRegression(), 479993.0, 'poly', 1]
modelo n° 30
el mejor modelo es el que minimiza el Mean Squared Error

{'modelo': LinearRegression(), 'puntaje': 479993.0, 'tipo': 'poly', 'grado': 1}


,predicted,actual
264,1487.735003,876
129,2408.735003,2484
179,768.735003,639
122,506.735003,807
152,1338.735003,916
37,1803.735003,1734
39,1426.735003,944


MSE: 479993.0

guardar el mejor modelo para usarlo en el futuro? no


In [178]:
modelosTodos[15][0]

[DecisionTreeRegressor(max_depth=1), 1457128.75, 'forest', 0]

In [175]:
from sklearn.metrics import mean_absolute_error

In [174]:
mean_absolute_error(outputTest,mejorModelo["modelo"].predict(PolynomialFeatures(degree=1).fit_transform(inputTest)))

483.1058522073261

In [176]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [177]:
MAPE(outputTest,mejorModelo["modelo"].predict(PolynomialFeatures(degree=1).fit_transform(inputTest)))

likes    60.332758
dtype: float64

In [69]:
from joblib import dump, load

In [135]:
dump(modelosTodos[27][0], 'Catriel_MejorModelo_Likes_decisionTree.joblib')

['Catriel_MejorModelo_Likes_decisionTree.joblib']

In [136]:
modelosTodos[27]

[RandomForestClassifier(max_depth=13), 0.6206896551724138, 'forest', 0]

In [137]:
pandas_puntajeTipoGrado=pd.DataFrame([modelosTodos[27][1], modelosTodos[27][2], modelosTodos[27][3]], columns=["Puntaje"])
pandas_puntajeTipoGrado.to_csv("Catriel_MejorModelo_Likes_info.csv")

In [138]:
pandas_ind = pd.DataFrame(ind)

In [139]:
pandas_ind.to_csv("Catriel_MejorModelo_Likes_variablesInd.csv")

In [153]:
mejorModelo={}
mejorModelo["puntaje"] = pd.read_csv("Catriel_MejorModelo_Likes_info.csv")["Puntaje"][0]
mejorModelo["tipo"] = pd.read_csv("Catriel_MejorModelo_Likes_info.csv")["Puntaje"][1]
mejorModelo["grado"] = pd.read_csv("Catriel_MejorModelo_Likes_info.csv")["Puntaje"][2]
mejorModelo["modelo"] = load('Catriel_MejorModelo_Likes_decisionTree.joblib')

In [141]:
scaler

StandardScaler()

In [142]:
dump(scaler, open('Catriel_MejorModelo_Likes_scaler.pkl', 'wb'))

In [64]:
from pickle import load

In [87]:
scaler = load(open('Catriel_MejorModelo_scaler.pkl', 'rb'))

In [143]:
ind

['agno',
 'mes',
 'dia',
 'hora',
 'len_caption',
 'es_video',
 'tiene_audio',
 'height',
 'width',
 'tag_users',
 'cantidad_hash',
 'ubicacion',
 'dia_semana - domingo',
 'dia_semana - jueves',
 'dia_semana - lunes',
 'dia_semana - martes',
 'dia_semana - miercoles',
 'dia_semana - sabado',
 'dia_semana - viernes']

In [156]:
confusion(mejorModelo["modelo"],int(mejorModelo["grado"]))

[[77 54]
 [45 85]]
acierta 0.59 de los negativos
acierta 0.65 de los positivos
0.63 de los predichos como negativos estan okey
0.61 de los predichos como positivos estan okey



In [133]:
modelosTodos[27]

[RandomForestClassifier(max_depth=13), 0.6206896551724138, 'forest', 0]

In [108]:
mejorModelo["modelo"].save('Catriel_MejorModelo_Likes_Deep.h5')

In [161]:
mejorModelo

{'modelo': LinearRegression(), 'puntaje': 479993.0, 'tipo': 'poly', 'grado': 1}

In [151]:
modelosTodos[27]

[RandomForestClassifier(max_depth=13), 0.6206896551724138, 'forest', 0]

## Pandora

In [69]:
for i in range(48,101,1):
    print("-----------------------------------------")
    print("navaja:",i/100)
    print(accuPandora(i/100))

-----------------------------------------
navaja: 0.48
[DecisionTreeClassifier(max_depth=1), 0.5104712041884817, 'tree', 0]
[DecisionTreeClassifier(max_depth=2), 0.5, 'tree', 0]
[DecisionTreeClassifier(max_depth=3), 0.5, 'tree', 0]
[DecisionTreeClassifier(max_depth=4), 0.5026178010471204, 'tree', 0]
[DecisionTreeClassifier(max_depth=6), 0.5104712041884817, 'tree', 0]
[DecisionTreeClassifier(max_depth=7), 0.5314136125654451, 'tree', 0]
[DecisionTreeClassifier(max_depth=8), 0.5287958115183246, 'tree', 0]
[DecisionTreeClassifier(max_depth=9), 0.5078534031413613, 'tree', 0]
[DecisionTreeClassifier(max_depth=10), 0.5392670157068062, 'tree', 0]
[DecisionTreeClassifier(max_depth=11), 0.5445026178010471, 'tree', 0]
[DecisionTreeClassifier(max_depth=12), 0.5314136125654451, 'tree', 0]
[RandomForestClassifier(max_depth=1), 0.5157068062827225, 'forest', 0]
[RandomForestClassifier(max_depth=2), 0.5418848167539267, 'forest', 0]
[RandomForestClassifier(max_depth=3), 0.5130890052356021, 'forest', 0]


<ipython-input-66-d731a1e8c9a3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandorasOutput[i][j]=float(k)


[0.48, 0.5104712041884817]
-----------------------------------------
navaja: 0.49
[DecisionTreeClassifier(max_depth=1), 0.5104712041884817, 'tree', 0]
[DecisionTreeClassifier(max_depth=2), 0.5, 'tree', 0]
[DecisionTreeClassifier(max_depth=3), 0.5, 'tree', 0]
[DecisionTreeClassifier(max_depth=4), 0.5026178010471204, 'tree', 0]
[DecisionTreeClassifier(max_depth=6), 0.5104712041884817, 'tree', 0]
[DecisionTreeClassifier(max_depth=7), 0.5314136125654451, 'tree', 0]
[DecisionTreeClassifier(max_depth=8), 0.5287958115183246, 'tree', 0]
[DecisionTreeClassifier(max_depth=9), 0.5078534031413613, 'tree', 0]
[DecisionTreeClassifier(max_depth=10), 0.5392670157068062, 'tree', 0]
[DecisionTreeClassifier(max_depth=11), 0.5445026178010471, 'tree', 0]
[DecisionTreeClassifier(max_depth=12), 0.5314136125654451, 'tree', 0]
[RandomForestClassifier(max_depth=1), 0.5157068062827225, 'forest', 0]
[RandomForestClassifier(max_depth=2), 0.5418848167539267, 'forest', 0]
[RandomForestClassifier(max_depth=3), 0.5130

<ipython-input-66-d731a1e8c9a3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandorasOutput[i][j]=float(k)


[0.49, 0.5497382198952879]
-----------------------------------------
navaja: 0.5
[DecisionTreeClassifier(max_depth=1), 0.5104712041884817, 'tree', 0]
[DecisionTreeClassifier(max_depth=4), 0.5026178010471204, 'tree', 0]
[DecisionTreeClassifier(max_depth=6), 0.5104712041884817, 'tree', 0]
[DecisionTreeClassifier(max_depth=7), 0.5314136125654451, 'tree', 0]
[DecisionTreeClassifier(max_depth=8), 0.5287958115183246, 'tree', 0]
[DecisionTreeClassifier(max_depth=9), 0.5078534031413613, 'tree', 0]
[DecisionTreeClassifier(max_depth=10), 0.5392670157068062, 'tree', 0]
[DecisionTreeClassifier(max_depth=11), 0.5445026178010471, 'tree', 0]
[DecisionTreeClassifier(max_depth=12), 0.5314136125654451, 'tree', 0]
[RandomForestClassifier(max_depth=1), 0.5157068062827225, 'forest', 0]
[RandomForestClassifier(max_depth=2), 0.5418848167539267, 'forest', 0]
[RandomForestClassifier(max_depth=3), 0.5130890052356021, 'forest', 0]
[RandomForestClassifier(max_depth=8), 0.5261780104712042, 'forest', 0]
[RandomFore

<ipython-input-66-d731a1e8c9a3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandorasOutput[i][j]=float(k)


[0.5, 0.5602094240837696]
-----------------------------------------
navaja: 0.51
[DecisionTreeClassifier(max_depth=1), 0.5104712041884817, 'tree', 0]
[DecisionTreeClassifier(max_depth=6), 0.5104712041884817, 'tree', 0]
[DecisionTreeClassifier(max_depth=7), 0.5314136125654451, 'tree', 0]
[DecisionTreeClassifier(max_depth=8), 0.5287958115183246, 'tree', 0]
[DecisionTreeClassifier(max_depth=10), 0.5392670157068062, 'tree', 0]
[DecisionTreeClassifier(max_depth=11), 0.5445026178010471, 'tree', 0]
[DecisionTreeClassifier(max_depth=12), 0.5314136125654451, 'tree', 0]
[RandomForestClassifier(max_depth=1), 0.5157068062827225, 'forest', 0]
[RandomForestClassifier(max_depth=2), 0.5418848167539267, 'forest', 0]
[RandomForestClassifier(max_depth=3), 0.5130890052356021, 'forest', 0]
[RandomForestClassifier(max_depth=8), 0.5261780104712042, 'forest', 0]
[RandomForestClassifier(max_depth=10), 0.5314136125654451, 'forest', 0]
[RandomForestClassifier(max_depth=11), 0.5157068062827225, 'forest', 0]
[Rand

<ipython-input-66-d731a1e8c9a3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandorasOutput[i][j]=float(k)


KeyboardInterrupt: 

In [129]:
navaja=float(input("a partir de que accuracy/mse quiere considerar un modelo como bueno? "))

a partir de que accuracy/mse quiere considerar un modelo como bueno? 10


In [142]:
accuPandora(navaja)

In [141]:
def accuPandora(navaja):
    global raza, inputTest
    aa=0
    pandorasOutput={}
    for i in modelosTodos:
        if not(math.isnan(i[1])):
            if ((raza == "regresion arbitraria") or (raza == "regresion porcentual") or (raza == "regresion positiva")) and (i[1] < navaja):
                pandorasOutput[aa] = list(pando(i[0],i[3],inputTest))
                print(i)
            elif (raza == "clasificacion binaria") and (i[1] > navaja):
                pandorasOutput[aa] = list(pando(i[0],i[3],inputTest))
                print(i)
        aa=aa+1
    pandorasOutput=pd.DataFrame.from_dict(pandorasOutput)
    for i in pandorasOutput:
        for j, k in enumerate(pandorasOutput[i]):
            pandorasOutput[i][j]=float(k)
    pandoraResultados=[]
    for i in range(len(pandorasOutput)):
        pandoraResultados.append(sum(pandorasOutput.loc[i])/len(pandorasOutput.loc[i]))
    if len(pandoraResultados) > 0:
        return [navaja, accuracy_score(outputTest.reset_index(drop=True),((pd.DataFrame(pandoraResultados)>0.5)*1))]

In [121]:
pandoraResultados=[]
for i in range(len(pandorasOutput)):
    pandoraResultados.append(sum(pandorasOutput.loc[i])/len(pandorasOutput.loc[i]))
pandoraResultados

NameError: name 'pandorasOutput' is not defined

In [40]:
accuracy_score(outputTest.reset_index(drop=True),((pd.DataFrame(pandoraResultados)>0.5)*1))

0.5523560209424084

In [105]:
MeanSquaredError()(outputTest.reset_index(drop=True),pd.DataFrame(pandoraResultados)).numpy()

1880.3756103515625

## Borrar Modelos o Verlos

In [145]:
modelosTodos=[]

In [42]:
pd.set_option('display.max_rows', pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"]).shape[0]+1)
pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])

,model,mse/accuracy,type,grade
0,DecisionTreeRegressor(max_depth=1),2.144319e+03,tree,0
1,DecisionTreeRegressor(max_depth=2),2.205400e+03,tree,0
2,DecisionTreeRegressor(max_depth=3),2.256011e+03,tree,0
3,DecisionTreeRegressor(max_depth=4),2.322245e+03,tree,0
4,DecisionTreeRegressor(max_depth=5),2.393752e+03,tree,0
5,DecisionTreeRegressor(max_depth=6),2.472747e+03,tree,0
6,DecisionTreeRegressor(max_depth=7),2.581198e+03,tree,0
7,DecisionTreeRegressor(max_depth=8),2.786563e+03,tree,0
8,DecisionTreeRegressor(max_depth=9),3.064584e+03,tree,0
9,DecisionTreeRegressor(max_depth=10),3.260248e+03,tree,0


## Corre Tree

In [38]:
for i in range(1,min(len(ind),15)+1):
    print("")
    print("###################################################################")
    print("----------------------------Model n°", (len(modelosTodos)), "----------------------------")
    print("###################################################################")
    modelosTodos.append(runCTree(i))
    print("")


###################################################################
----------------------------Model n° 24 ----------------------------
###################################################################
accuracy score: 0.5148


,pred-gender - Female,pred-gender - Male,pred-gender - Other,pred-ever_married - No,pred-ever_married - Yes,res-gender - Female,res-gender - Male,res-gender - Other,res-ever_married - No,res-ever_married - Yes,act-gender - Female,act-gender - Male,act-gender - Other,act-ever_married - No,act-ever_married - Yes
420,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
35,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
1019,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
620,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0
491,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0
867,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
1073,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1


raza = multiple label


###################################################################
----------------------------Model n° 25 ----------------------------
###################################################################
accuracy score: 0.5148


,pred-gender - Female,pred-gender - Male,pred-gender - Other,pred-ever_married - No,pred-ever_married - Yes,res-gender - Female,res-gender - Male,res-gender - Other,res-ever_married - No,res-ever_married - Yes,act-gender - Female,act-gender - Male,act-gender - Other,act-ever_married - No,act-ever_married - Yes
1071,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
1610,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
1375,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
1356,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
697,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0
1420,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
1575,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1


raza = multiple label


###################################################################
----------------------------Model n° 26 ----------------------------
###################################################################
accuracy score: 0.5148


...

raza = multiple label


###################################################################
----------------------------Model n° 28 ----------------------------
###################################################################
accuracy score: 0.5148


,pred-gender - Female,pred-gender - Male,pred-gender - Other,pred-ever_married - No,pred-ever_married - Yes,res-gender - Female,res-gender - Male,res-gender - Other,res-ever_married - No,res-ever_married - Yes,act-gender - Female,act-gender - Male,act-gender - Other,act-ever_married - No,act-ever_married - Yes
1554,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0
176,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0
838,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0
1169,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0
1406,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0
387,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0
192,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1


raza = multiple label


###################################################################
----------------------------Model n° 29 ----------------------------
###################################################################
accuracy score: 0.5185


,pred-gender - Female,pred-gender - Male,pred-gender - Other,pred-ever_married - No,pred-ever_married - Yes,res-gender - Female,res-gender - Male,res-gender - Other,res-ever_married - No,res-ever_married - Yes,act-gender - Female,act-gender - Male,act-gender - Other,act-ever_married - No,act-ever_married - Yes
616,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
225,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0
741,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
168,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
139,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
1232,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0
305,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1


raza = multiple label



## Corre Forest

In [39]:
for i in range(1,min(len(ind),15)+1):
    print("")
    print("###################################################################")
    print("----------------------------Model n°", (len(modelosTodos)), "----------------------------")
    print("###################################################################")
    modelosTodos.append(runRForest(i))
    print("")


###################################################################
----------------------------Model n° 30 ----------------------------
###################################################################
accuracy score: 0.4741


,pred-gender - Female,pred-gender - Male,pred-gender - Other,pred-ever_married - No,pred-ever_married - Yes,res-gender - Female,res-gender - Male,res-gender - Other,res-ever_married - No,res-ever_married - Yes,act-gender - Female,act-gender - Male,act-gender - Other,act-ever_married - No,act-ever_married - Yes
1176,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
382,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
1382,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0
1339,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
302,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
584,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0
621,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1


raza = multiple label


###################################################################
----------------------------Model n° 31 ----------------------------
###################################################################
accuracy score: 0.5154


,pred-gender - Female,pred-gender - Male,pred-gender - Other,pred-ever_married - No,pred-ever_married - Yes,res-gender - Female,res-gender - Male,res-gender - Other,res-ever_married - No,res-ever_married - Yes,act-gender - Female,act-gender - Male,act-gender - Other,act-ever_married - No,act-ever_married - Yes
1316,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0
1565,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
1081,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0
125,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
622,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0
953,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
758,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0


raza = multiple label


###################################################################
----------------------------Model n° 32 ----------------------------
###################################################################
accuracy score: 0.5154


...

raza = multiple label


###################################################################
----------------------------Model n° 34 ----------------------------
###################################################################
accuracy score: 0.5309


,pred-gender - Female,pred-gender - Male,pred-gender - Other,pred-ever_married - No,pred-ever_married - Yes,res-gender - Female,res-gender - Male,res-gender - Other,res-ever_married - No,res-ever_married - Yes,act-gender - Female,act-gender - Male,act-gender - Other,act-ever_married - No,act-ever_married - Yes
101,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1
858,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0
110,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1
48,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
1416,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0
662,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
408,1,0,0,0,1,1,0,0,0,1,0,1,0,1,0


raza = multiple label


###################################################################
----------------------------Model n° 35 ----------------------------
###################################################################
accuracy score: 0.5259


,pred-gender - Female,pred-gender - Male,pred-gender - Other,pred-ever_married - No,pred-ever_married - Yes,res-gender - Female,res-gender - Male,res-gender - Other,res-ever_married - No,res-ever_married - Yes,act-gender - Female,act-gender - Male,act-gender - Other,act-ever_married - No,act-ever_married - Yes
928,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0
371,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
568,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
671,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
1257,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0
1564,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
1458,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0


raza = multiple label



## Corre Poly

In [40]:
for i in range(1,min(len(ind),5)+1):
    print("")
    print("###################################################################")
    print("----------------------------Model n°", (len(modelosTodos)), "----------------------------")
    print("###################################################################")
    modelosTodos.append(runPoly(i))
    print("")


###################################################################
----------------------------Model n° 36 ----------------------------
###################################################################
coefficient of determination: 19.68 %
raza = multiple label


TypeError: type NoneType doesn't define __round__ method

## Elije el Modelo mas Efectivo

In [35]:
if (raza == "regresion arbitraria") or (raza == "regresion porcentual") or (raza == "regresion positiva"):
    print(modelosTodos[list(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"]).index(min(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"]))])
    print("modelo n°", list(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"]).index(min(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"])))
elif raza == "clasificacion binaria":
    print(modelosTodos[list(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"]).index(max(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"]))])
    print("modelo n°", list(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"]).index(max(pd.DataFrame(modelosTodos, columns=["model","mse/accuracy","type","grade"])["mse/accuracy"])))
else:
    print("raza =". raza)

[LinearRegression(), 1825.1571044921875, 'poly', 1]
modelo n° 56


## Diseñar Neural y correr todos

In [133]:
disenarModelos()
crearModelos()
print("hay un total de",len(models), "modelos")
print("")

print("-------------------------Modelos-------------------------")
if len(models)<10:
    for i in range(len(models)):
        print(models[i])
    print("")
else:
    for i in range(10):
        print(models[i])
    print("")

cuantas capas tendrá el modelo? (1, 2 o 3) 2
cuantas celulas habrá en la 1er capa? 4
activacion de la capa 1? (sigmoid,softmax,tanh,relu,softplus,softsign,selu,elu,exponential) sigmoid,softmax,tanh,relu
cuantas celulas habrá en la 2da capa? 4
activacion de la capa 2? (sigmoid,softmax,tanh,relu,softplus,softsign,selu,elu,exponential) sigmoid,softmax,tanh,relu
learning rate? (0.1, 0.01, 0.001, etc) 0.1, 0.01, 0.001

hay un total de 48 modelos

-------------------------Modelos-------------------------
(4, 'sigmoid', 4, 'sigmoid', 0, 0, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.1)
(4, 'sigmoid', 4, 'softmax', 0, 0, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.1)
(4, 'sigmoid', 4, 'tanh', 0, 0, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.1)
(4, 'sigmoid', 4, 'relu', 0, 0, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.1)
(4, 'softmax', 4, 'sigmoid', 0, 0, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.1)
(4, 'softmax

In [134]:
modelosTodos=[]
for i in range(len(models)):
    print("")
    print("###################################################################")
    print("----------------------------Model n°", i, "----------------------------")
    print("###################################################################")
    modelosTodos.append(run(models[i][0],models[i][1],models[i][2],models[i][3],models[i][4],models[i][5],models[i][6],models[i][7]))
    print("")


###################################################################
----------------------------Model n° 0 ----------------------------
###################################################################

4 sigmoid 4 sigmoid 0 0 <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'> 0.1



IndexError: tuple index out of range

In [145]:
len(modelosTodos)/3

24.0

In [146]:
adam=0
sgd=0
rms=0
for i, j in enumerate(modelosTodos):
    if i < 24:
        adam+=j[1]
    elif i < 48:
        sgd+=j[1]
    else:
        rms+=j[1]
adam=adam/24
sgd=sgd/24
rms=rms/24

In [147]:
print(adam)
print(sgd)
print(rms)

0.7210144927536232
0.7189009661835749
0.7155797101449274


## Mejor Modelo y Resultados (roto)

In [142]:
if (raza == "clasificacion binaria") or (raza == "multiple label") or (raza == "simple label"):
    mejorModelo={"modelo":0, "puntaje":0}
    print("el mejor modelo es el que aumenta la presición")
    print("")
    for i, j in modelosTodos:
        if j > mejorModelo["puntaje"]:
            mejorModelo["modelo"]=i
            mejorModelo["puntaje"]=j
else:
    mejorModelo={"modelo":999999999999999999, "puntaje":999999999999999999}
    print("el mejor modelo es el que minimiza el Mean Squared Error")
    print("")
    for i, j in modelosTodos:
        if j < mejorModelo["puntaje"]:
            mejorModelo["modelo"]=i
            mejorModelo["puntaje"]=j
print(mejorModelo)
print("")
print(mejorModelo["modelo"].summary())
print("")
if (raza == "clasificacion binaria") or (raza == "multiple label") or (raza == "simple label"):
    comparativo=pd.DataFrame(mejorModelo["modelo"].predict(inputTest), columns = dep)
    comparativo.columns = ["predicted"]
    comparativo["predicted"]=(mejorModelo["modelo"].predict(inputTest)>0.5)*1
    comparativo[dep]=outputTest.reset_index(drop=True, inplace=False)
    comparativo[dep]=comparativo[dep].fillna(0)
    comparativo[dep]=comparativo[dep].astype(int)
    display(comparativo.sample(7))
    print(confusion_matrix(comparativo[dep],comparativo["predicted"]))
    print("acierta", round(confusion_matrix(comparativo[dep],comparativo["predicted"])[0][0]/sum(confusion_matrix(comparativo[dep],comparativo["predicted"])[0]),2),"de los negativos")
    print("acierta", round(confusion_matrix(comparativo[dep],comparativo["predicted"])[1][1]/sum(confusion_matrix(comparativo[dep],comparativo["predicted"])[1]),2),"de los positivos")
    print(round(confusion_matrix(comparativo[dep],comparativo["predicted"])[0][0]/(confusion_matrix(comparativo[dep],comparativo["predicted"])[0][0]+confusion_matrix(comparativo[dep],comparativo["predicted"])[1][0]),2),"de los predichos como negativos estan okey")
    print(round(confusion_matrix(comparativo[dep],comparativo["predicted"])[1][1]/(confusion_matrix(comparativo[dep],comparativo["predicted"])[1][1]+confusion_matrix(comparativo[dep],comparativo["predicted"])[0][1]),2),"de los predichos como positivos estan okey")
    print("")
else:
    comparativo=pd.DataFrame(mejorModelo["modelo"].predict(inputTest), columns = dep)
    comparativo.columns = ["predicted"]
    comparativo["predicted"]=mejorModelo["modelo"].predict(inputTest)
    comparativo[dep]=outputTest.reset_index(drop=True, inplace=False)
    comparativo[dep]=comparativo[dep].fillna(0)
    comparativo[dep]=comparativo[dep].astype(int)
    display(comparativo.sample(7))
    mae=[]
    mse=[]
    for i in range(comparativo.shape[0]):
        ver=abs(comparativo["predicted"][i]-comparativo[comparativo.columns[1]][i])
        mae.append(ver)
        ver=ver*ver
        mse.append(ver)
    mae=sum(mae)/comparativo.shape[0]
    mse=sum(mse)/comparativo.shape[0]
    print("Mean Absolute Error:", round(mae,4))
    print("Mean Squared Error:", round(mse,4))
    print("")

el mejor modelo es el que aumenta la presición



ValueError: too many values to unpack (expected 2)

## Crea Neurals Automáticos y corre 25

In [124]:
layers=[1, 2, 3]
hidden1=[2, 4, 8, 16, 32, 64]
activation1=['sigmoid', 'softmax', 'tanh', 'relu', 'softplus', 'softsign', 'selu', 'elu', 'exponential']
hidden2=[2, 4, 8, 16, 32, 64]
activation2=['sigmoid', 'softmax', 'tanh', 'relu', 'softplus', 'softsign', 'selu', 'elu', 'exponential']
hidden3=[2, 4, 8, 16, 32, 64]
activation3=['sigmoid', 'softmax', 'tanh', 'relu', 'softplus', 'softsign', 'selu', 'elu', 'exponential']
optimizer=[Adam]
learningRate=[0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
crearModelos()
print("hay un total de",len(models), "modelos")
print("")
random.shuffle(models)
print("-------------------------Modelos-------------------------")
for i in range(15):
    print(models[i])
print("")

hay un total de 962604 modelos

-------------------------Modelos-------------------------
(2, 'sigmoid', 2, 'softmax', 8, 'softsign', <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.5)
(2, 'softmax', 32, 'tanh', 2, 'relu', <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.5)
(2, 'sigmoid', 16, 'tanh', 32, 'sigmoid', <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.005)
(4, 'softplus', 64, 'tanh', 2, 'sigmoid', <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.001)
(2, 'exponential', 2, 'sigmoid', 32, 'softsign', <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.05)
(4, 'elu', 16, 'sigmoid', 8, 'exponential', <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.5)
(4, 'selu', 16, 'selu', 32, 'sigmoid', <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.01)
(32, 'softplus', 64, 'softmax', 64, 'softsign', <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 0.05)
(8, 'softplus', 2, 'sigmoid', 8, 'tanh', <class 'tens

In [132]:
for i in range(25):
    print("")
    print("###################################################################")
    print("----------------------------Model n°", (len(modelosTodos)), "----------------------------")
    print("###################################################################")
    modelosTodos.append(run(models[i][0],models[i][1],models[i][2],models[i][3],models[i][4],models[i][5],models[i][6],models[i][7]))
    print("")


###################################################################
----------------------------Model n° 31 ----------------------------
###################################################################

2 sigmoid 2 softmax 8 softsign <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'> 0.5



IndexError: tuple index out of range

In [168]:
from tensorflow import keras

In [169]:
inputTrain.to_csv("g_inputTrain.csv")
inputTest.to_csv("g_inputTest.csv")
outputTrain.to_csv("g_outputTrain.csv")
outputTest.to_csv("g_outputTest.csv")

In [170]:
mejorModelo["modelo"].save('g_mejorModelo.h5')

In [171]:
pandas_puntaje=pd.DataFrame([mejorModelo["puntaje"]], columns=["Puntaje"])
pandas_puntaje.to_csv("g_puntajeMejorModelo.csv")

In [118]:
mejorModelo={}
mejorModelo["puntaje"] = pd.read_csv("g_puntajeMejorModelo.csv")
mejorModelo["puntaje"] = mejorModelo["puntaje"]["Puntaje"][0]

In [121]:
mejorModelo["modelo"] = keras.models.load_model('g_mejorModelo.h5')

In [13]:
inputTrain=pd.read_csv("g_inputTrain.csv",index_col=0)
inputTest=pd.read_csv("g_inputTest.csv",index_col=0)
outputTrain=pd.read_csv("g_outputTrain.csv",index_col=0)
outputTest=pd.read_csv("g_outputTest.csv",index_col=0)

In [27]:
unos=0
ceros=0
for index, i in data.iterrows():
    if i["close2"] == 0:
        ceros+=1
        if ceros > minimo:
            data = data.drop([index], axis=0)
    else:
        unos+=1
        if unos > minimo:
            data = data.drop([index], axis=0)


In [30]:
variables=[]
for i in data:
    variables.append(i)
elegirVariables()
standarizarData()

puede definir variables independientes y dependientes entre las siguientes:
['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'high2', 'low2', 'close2']
high2 es una variable independiente
low2 es una variable independiente
close2 es una variable dependiente
Standarizar data? no
okey
